In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

FILE = "data/database1.csv"
RANDOM_STATE = 42

df = pd.read_csv(FILE)

# X_train, X_test, y_train, y_test = train_test_split(
#     df["title"], df["visited"], test_size=0.33, random_state=RANDOM_STATE)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape
df

# df = pd.read_csv(FILE)
# print(df.columns)
# # df = df[["title", "points", "comments", "viewed"]]
# # df["viewed"] = df["viewed"].apply((lambda x: 1 if x==1 else 0))
# # df.loc[df["viewed"].isna(), "viewed"] = 0
# # df.loc[[1,3,10,12,18,19,21],"viewed"] = 1
# df

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,title,time,url,visited
0,0,0,0,0,0,0,Jraft – the best blogging platform ever,2020-11-16 02:12:45,http://www.tryjraft.com,1
1,1,1,1,1,1,1,"California, Love It and Leave It",2020-11-16 01:33:38,https://www.wsj.com/articles/california-love-i...,0
2,2,2,2,2,2,2,"SpaceX Starlink has some hiccups as expected, ...",2020-11-16 00:56:03,https://arstechnica.com/information-technology...,1
3,3,3,3,3,3,3,SpaceX Crew-1 [video],2020-11-16 00:23:38,https://www.youtube.com/watch?v=bnChQbxLkkI&re...,1
4,4,4,4,4,4,4,The Treehouse Is De-Plaformed,2020-11-15 23:28:32,https://theconservativetreehouse.com/2020/11/1...,0
...,...,...,...,...,...,...,...,...,...,...
1487,1487,1487,1487,1487,1487,1487,U.S. Feds Seized Nearly $1B in Bitcoin from Wa...,2020-11-05 18:22:07,https://www.vice.com/en/article/akdgz8/us-feds...,0
1488,1488,1488,1488,1488,1488,1488,Facebook was used as a proxy by web scraping bots,2020-11-05 18:15:42,https://datadome.co/bot-detection/how-facebook...,1
1489,1489,1489,1489,1489,1489,1489,How is Visual Basic still ranked #6 programmin...,2020-11-05 18:06:53,https://www.tiobe.com/tiobe-index/,0
1490,1490,1490,1490,1490,1490,1490,Confessions of a voter fraud: I was a master a...,2020-11-05 17:48:36,https://nypost.com/2020/08/29/political-inside...,0


In [2]:
import numpy as np
import torch
import transformers as ppb

pretrained_weights_name = "distilbert-base-uncased"
# pretrained_weights_name = "bert-base-uncased"
tokenizer = ppb.DistilBertTokenizer.from_pretrained(pretrained_weights_name)
model = ppb.DistilBertModel.from_pretrained(pretrained_weights_name)

tokenized = df["title"].apply((lambda s: tokenizer.encode(s, add_special_tokens=True)))

max_len = max([len(row) for row in tokenized])
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

print(padded.shape)
padded

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1492, 31)


array([[  101,  3781, 10354, ...,     0,     0,     0],
       [  101,  2662,  1010, ...,     0,     0,     0],
       [  101,  2686,  2595, ...,     0,     0,     0],
       ...,
       [  101,  2129,  2003, ...,     0,     0,     0],
       [  101, 21444,  1997, ...,     0,     0,     0],
       [  101,  1996,  3010, ...,     0,     0,     0]])

In [3]:
encoded = torch.tensor(padded)
attention_mask = np.where(padded != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(encoded, attention_mask=attention_mask)

# last_hidden_states[0].shape



features = last_hidden_states[0][:,0,:].numpy()
print(features.shape)
labels = df["visited"].to_list()

# # add features
# new_features = ["points","comments"]
# features = np.append(features, df[new_features].to_numpy(), axis=1)
# print(features.shape)

# features, labels

(1492, 768)


In [4]:
features = last_hidden_states[0][:,0,:].numpy()
# labels = df["viewed"]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, random_state=RANDOM_STATE)

from sklearn.linear_model import LogisticRegression, LinearRegression
clf = LogisticRegression(random_state=RANDOM_STATE)
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))

from sklearn.metrics import roc_auc_score
y_score = clf.predict_proba(x_test)[:,1]
print(roc_auc_score(y_test, y_score))

# lin_clf.fit(x_train, y_train)
# print(lin_clf.score(x_test, y_test))

0.8194726166328601
0.6140570934256055


/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
clf = DummyClassifier()
scores = cross_val_score(clf, x_train, y_train)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.829 (+/- 0.00)
